<a href="https://colab.research.google.com/github/chodizzle/portfolio1/blob/main/projects/serious_eats_recipe_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [ ]:
%cd drive/MyDrive/Colab\ Notebooks/serious_recipes

/content/drive/MyDrive/Colab Notebooks/serious_recipes


In [ ]:
!pwd
%ls

/content/drive/MyDrive/Colab Notebooks/serious_recipes
recipes.db


In [ ]:
conn = sqlite3.connect('recipes.db')
cur = conn.cursor()

Create tables then comment out for recipes

In [ ]:
# cur.executescript('''
# drop table if exists Recipes;
# drop table if exists Rec_Procedures;
# drop table if exists Rec_Ingredients;
# drop table if exists Ingredients;
# drop table if exists Ingredient_Category;
# drop table if exists Link;
# ''')

In [ ]:
cur.executescript('''
CREATE TABLE Recipes(
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
  name TEXT,
  source TEXT,
  prep_time TEXT,
  total_time TEXT,
  rating_5 REAL,
  yield TEXT
);

CREATE TABLE Rec_Procedures (
  recipe_id INTEGER NOT NULL,
  procedure_num INTEGER NOT NULL,
  procedure_text TEXT,
  PRIMARY KEY (recipe_id, procedure_num),
  FOREIGN KEY (recipe_id) REFERENCES Recipes (id)
  ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE Rec_Ingredients (
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
  recipe_id INTEGER NOT NULL,
  ingredient_id INTEGER NOT NULL,
  quantity REAL,
  UOM TEXT,
  prep TEXT,
  quantity_2 REAL,
  UOM_2 TEXT
);

CREATE TABLE Ingredients (
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
  name TEXT,
  cat_id INTEGER
);

CREATE TABLE Ingredient_Category (
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
  name TEXT
);

CREATE TABLE Links (
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
  link_name TEXT UNIQUE,
  visited BOOLEAN  
);

''')

In [ ]:
# cur.execute("Select * from sqlite_master where type='table';")
# tables = cur.fetchall()
# for x in tables:
#   x = x[1]
#   table = pd.read_sql_query("select * from %s" % x, conn)
#   print(table)

In [ ]:
cur.execute("select name from sqlite_master where type='table'")
print(cur.fetchall())

[('Recipes',), ('sqlite_sequence',), ('Rec_Procedures',), ('Rec_Ingredients',), ('Ingredients',), ('Ingredient_Category',), ('Links',)]


In [ ]:
cur.execute('''select * from Recipes''')
for row in cur:
  print(row)

the code below is for getting links to recipes on the Serious Eats Website

In [ ]:
links = []
for i in range(1, 12):
  rec = requests.get('https://www.seriouseats.com/recipes/topics/meal/quick-dinners?page={}#recipes'.format(i)).text
  supersoup = BeautifulSoup(rec, 'html.parser')
  recipes = supersoup.find_all('h4', {'class':'c-card__title'})
  
  templinks = []
  for recipe in recipes:
    link = recipe.a['href']
    if link in links: continue
    else: links.append(link)  

print(len(links))

0


In [ ]:
for link in links:
  recdet = requests.get(link).text
  deets = BeautifulSoup(recdet, 'html.parser')

  try:
  name = deets.find('h1', {'class':'title recipe-title c-post__h1'}).text.strip()
  except:
    name = None

  try:
    abouts = deets.find('ul', {'class':'recipe-about'})
    about_labels = abouts.find_all('span', {'class':'label'})
    about_values = abouts.find_all('span', {'class':'info'})

    del about_values[-1]

    about = {}

    if len(about_labels) == len(about_values):
      for i in range(len(about_labels)):
        label = about_labels[i].text.strip()
        value = about_values[i].text.strip()
        about[label] = value
    else:
      about = None
    
  except:
    about = None

  try:
    ingredients = deets.find('div', {'class':'recipe-ingredients'}).text
  except:
    ingredients = None

  try:
    procedure_no = deets.find_all('div', {'class':'recipe-procedure-number'})
    procedure_text = deets.find_all('div', {'class':'recipe-procedure-text'})
    procedure = {}
    if len(procedure_no) == len(procedure_text):
      for i in range(len(procedure_no)):
        proc_no = procedure_no[i].text
        proc_text = procedure_text[i].text.strip()
        procedure[proc_no] = proc_text
    else:
      procedure = None
  except:
    procedure = None

  recipe = [name, about, ingredients, procedure]



In [ ]:
recdet = requests.get('https://www.seriouseats.com/recipes/2017/07/peruvian-style-grilled-chicken-sandwiches-recipe.html').text
deets = BeautifulSoup(recdet, 'html.parser')

try:
  name = deets.find('h1', {'class':'title recipe-title c-post__h1'}).text.strip()
except:
  name = None

try:
  abouts = deets.find('ul', {'class':'recipe-about'})
  about_labels = abouts.find_all('span', {'class':'label'})
  about_values = abouts.find_all('span', {'class':'info'})

  del about_values[-1]

  about = {}

  if len(about_labels) == len(about_values):
    for i in range(len(about_labels)):
      label = about_labels[i].text.strip()
      value = about_values[i].text.strip()
      about[label] = value
  else:
    about = None
  
except:
  about = None

try:
  ingredients = deets.find('div', {'class':'recipe-ingredients'}).text
except:
  ingredients = None

try:
  procedure_no = deets.find_all('div', {'class':'recipe-procedure-number'})
  procedure_text = deets.find_all('div', {'class':'recipe-procedure-text'})
  procedure = {}
  if len(procedure_no) == len(procedure_text):
    for i in range(len(procedure_no)):
      proc_no = procedure_no[i].text
      proc_text = procedure_text[i].text.strip()
      procedure[proc_no] = proc_text
  else:
    procedure = None
except:
  procedure = None

recipe = [name, about, ingredients, procedure]

for x in recipe:
  if type(x) == dict:
    for k,v in x.items():
      print(k,v)
  else:
    print(x.strip())


Peruvian-Style Grilled-Chicken Sandwiches With Spicy Green Sauce Recipe
Yield: Serves 4
Active time: 1 hour
Total time: 1 hour
Rated: 4.5
Ingredients

For the Chicken:
3 medium cloves garlic, roughly chopped
4 teaspoons (15g) kosher salt
2 tablespoons ground cumin
2 tablespoons paprika
1 teaspoon freshly ground black pepper
2 tablespoons (30ml) white vinegar 
2 tablespoons (30ml) vegetable or canola oil
4 boneless, skinless chicken breasts (5 to 7 ounces/140 to 200g each), cut into 8 cutlets
For the Sauce:
3 whole jalapeño chilies, roughly chopped (see note)
1 tablespoon (15ml) ají amarillo pepper paste (optional; see note)
1 cup (1 ounce) fresh cilantro leaves
2 medium cloves garlic
1/2 cup (120ml) mayonnaise 
1/4 cup (60ml) sour cream
2 teaspoons (10ml) fresh juice from 1 lime
1 teaspoon (5ml) distilled white vinegar 
1/4 cup (60ml) extra-virgin olive oil
Kosher salt and freshly ground black pepper
To Serve:
Four sturdy buns, such as telera or ciabatta
2 ripe avocados, halved, pitted

In [ ]:
cur.execute('''insert into Recipes (name, about, ingredients, procedure) values (?, ?, ?, ?)''', (name, 'blah', ingredients, procedure.values()))

In [ ]:
cur.execute('''select * from Recipes''')
print(cur.fetchall())

[(1, 'blah', 'blah', 'blah', 'blah'), (2, 'Peruvian-Style Grilled-Chicken Sandwiches With Spicy Green Sauce Recipe', 'blah', '\nIngredients\n\nFor the Chicken:\n3 medium cloves garlic, roughly chopped\n4 teaspoons (15g) kosher salt\n2 tablespoons ground cumin\n2 tablespoons paprika\n1 teaspoon freshly ground black pepper\n2 tablespoons (30ml) white vinegar \n2 tablespoons (30ml) vegetable or canola oil\n4 boneless, skinless chicken breasts (5 to 7 ounces/140 to 200g each), cut into 8 cutlets\nFor the Sauce:\n3 whole jalapeño chilies, roughly chopped (see note)\n1 tablespoon (15ml) ají amarillo pepper paste (optional; see note)\n1 cup (1 ounce) fresh cilantro leaves\n2 medium cloves garlic\n1/2 cup (120ml) mayonnaise \n1/4 cup (60ml) sour cream\n2 teaspoons (10ml) fresh juice from 1 lime\n1 teaspoon (5ml) distilled white vinegar \n1/4 cup (60ml) extra-virgin olive oil\nKosher salt and freshly ground black pepper\nTo Serve:\nFour sturdy buns, such as telera or ciabatta\n2 ripe avocados, 